# Charger les données

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
'FDS = Fragrantica Data Set'
FDS = pd.read_csv('Fragrantica_dataset.csv')

'''ATTENTION: Ici on prend uniquement un petit sample du dataset pour
pouvoir le manipuler plus facilement.'''
#FDS = FDS.sample(100, random_state=1)
FDS.head()

,nom_parfum,marque,nose,launch_year,rating_value,rating_count,main_accords,gender,longevity,sillage,price_feeling,top_notes,middle_notes,base_notes,url
0,#Cool Vibes,Zara,NaN,2020.0,4.23,13,"['vanilla', 'fruity', 'sweet', 'citrus', 'powd...",female,very weak,NaN,ok,[],[],[],https://www.fragrantica.com/perfume/Zara/Cool-...
1,#Weekend Plan,Zara,NaN,2020.0,4.33,3,"['citrus', 'floral', 'fresh spicy', 'vanilla',...",NaN,NaN,NaN,NaN,[],[],[],https://www.fragrantica.com/perfume/Zara/Weeke...
2,01 Magic Onsen,Zara,NaN,2017.0,3.29,38,"['musky', 'white floral', 'woody', 'powdery', ...",NaN,very weak,intimate,NaN,[],[],[],https://www.fragrantica.com/perfume/Zara/01-Ma...
3,02 Sweet Vanilla,Zara,NaN,2017.0,3.93,127,"['fruity', 'powdery', 'vanilla', 'woody', 'swe...",female,moderate,average,good value,"['Chestnut', 'Almond']",['Caramel'],"['Cedar', 'Vanilla', 'Musk']",https://www.fragrantica.com/perfume/Zara/02-Sw...
4,Amber Vanilla,Zara,NaN,NaN,4.23,35,"['amber', 'vanilla', 'powdery']",NaN,long lasting,average,NaN,[],[],[],https://www.fragrantica.com/perfume/Zara/Amber...


# Nettoyage données

On veut que rating_count soit un entier

In [3]:
FDS['rating_count'] = FDS['rating_count'].fillna(0)
FDS['rating_count'] = FDS['rating_count'].astype(str).str.replace(',', '').str.replace('.0', '')
FDS['rating_count'] = FDS['rating_count'].astype(int)

## Gestion données manquantes

In [4]:
# Détection des valeurs manquantes et proportion de valeurs manquantes
print(f"Valeurs manquantes = {FDS.isnull().sum()}")
pourcentage = FDS.isnull().sum() / FDS.shape[0] * 100
print(f"Pourcentage de valeurs manquantes = {pourcentage} %")
# Pour l'instant on garde tout


Valeurs manquantes = nom_parfum        0
marque            0
nose             63
launch_year       9
rating_value      4
rating_count      0
main_accords      0
gender           34
longevity        25
sillage          23
price_feeling    36
top_notes         0
middle_notes      0
base_notes        0
url               0
dtype: int64
Pourcentage de valeurs manquantes = nom_parfum        0.0
marque            0.0
nose             63.0
launch_year       9.0
rating_value      4.0
rating_count      0.0
main_accords      0.0
gender           34.0
longevity        25.0
sillage          23.0
price_feeling    36.0
top_notes         0.0
middle_notes      0.0
base_notes        0.0
url               0.0
dtype: float64 %


**Il y 1920 années de lancement manquantes** ce n'est pas un pb de scrapping mais bien d'info manquante sur le site

In [5]:
# Affichons quelques parfums ou lunch_year est manquant
annee_manquante = FDS[FDS['launch_year'].isnull()]
#annee_manquante.to_csv('annee_manquante.csv', index=False)


## Autres modifications

**Doublons**

In [6]:
#Repérage des doublons
print(f"Nombre de doublons = {FDS['url'].duplicated().sum()}")


Nombre de doublons = 0


**mettre nom des varialbes an anglais**

In [7]:
# On met tous les nomq de variables en anglais
FDS = FDS.rename(columns={'marque': 'brand'})
FDS = FDS.rename(columns={'nom_parfum': 'name'})

Mettre les 'nose' vide en str

In [8]:
# Transformer toute la colonne 'nose' en type str
FDS['nose'] = FDS['nose'].astype(str)

# Transformation données

## Regrouper les modalités rares de brand et nose dans (« Autres »)

On garde les **20** premiers

In [9]:
def keep_top_k(series, k=20):
    top = series.value_counts().nlargest(k).index
    return series.where(series.isin(top), other="Other")
FDS["brand_reduce"] = keep_top_k(FDS["brand"], 20)
FDS["nose_reduce"]   = keep_top_k(FDS["nose"],   20)

FDS.head()


,name,brand,nose,launch_year,rating_value,rating_count,main_accords,gender,longevity,sillage,price_feeling,top_notes,middle_notes,base_notes,url,brand_reduce,nose_reduce
2526,Love Spell Radiant,Victoria's Secret,nan,2021.0,3.96,25,"['tropical', 'citrus', 'sweet', 'fruity', 'ter...",female,weak,average,good value,[],[],[],https://www.fragrantica.com/perfume/Victoria-s...,Victoria's Secret,nan
14959,Fresh Sparkling Snow,Bath & Body Works,nan,2015.0,4.10,82,"['fresh', 'fruity', 'aquatic', 'ozonic', 'flor...",female,moderate,average,good value,[],[],[],https://www.fragrantica.com/perfume/Bath-Body-...,Bath & Body Works,nan
13474,Hugo Urban Journey,Hugo Boss,nan,2018.0,3.65,142,"['green', 'woody', 'floral', 'fresh']",male,moderate,average,ok,"['Green Accord', 'Floral Notes']",['Black Tea'],"['Guaiac Wood', 'Sandalwood']",https://www.fragrantica.com/perfume/Hugo-Boss/...,Other,nan
7502,Dr. Botica Poção da Coragem,O Boticário,nan,2018.0,3.90,29,"['aromatic', 'marine', 'fresh spicy', 'woody',...",unisex,moderate,intimate,good value,"['Fennel', 'Rosemary', 'Citruses']","['Sea Notes', 'Woodsy Notes']","['Amber', 'Musk']",https://www.fragrantica.com/perfume/O-Boticari...,O Boticário,nan
12542,Idylle,Guerlain,Thierry Wasser,2009.0,3.79,6705,"['white floral', 'floral', 'fresh', 'rose', 'm...",female,moderate,average,ok,"['Raspberry', 'Litchi', 'Freesia', 'Tincture o...","['Lily-of-the-Valley', 'Lilac', 'Peony', 'Jasm...","['Musk', 'Patchouli']",https://www.fragrantica.com/perfume/Guerlain/I...,Guerlain,Other


##  Segmentation des variables numériques
   - Séparer l’année de lancement en périodes (ex. avant 2000, 2000-2010, etc.) pour des analyses comparatives.
   - Catégoriser ou agréger certaines variables si elles sont trop granulaires.

In [10]:
# name --> OK
# brand --> Segmentention possibles : , par nombre de parfum, Par popularité, par type de parfum, par genre, par prix, ...
# nose --> Segmentation possibles : par nombre de parfum, par popularité, par genre, par prix, ...

#launch_year --> Créer une variable 'launch_period' par groupe de 5 année
FDS['launch_period'] = pd.cut(FDS['launch_year'],
                              bins=[1900, 1995,2000,2005,2010,2015,2020,2030],
                              labels=['Avant 1995', '1995-2000', '2000-2005', '2005-2010', '2010-2015', '2015-2020', 'Après 2020'])

# rating value --> Créer une variable 'rating_category' en 3 catégories || On sépare après avoir regardé distrubtion des notes
FDS['rating_value_category'] = pd.cut(FDS['rating_value'],
                                bins=[0, 3.3, 3.9, 4.2, 4.5, 5],
                                labels=['Très faible [0 , 3.3]', 'Faible ]3.3 , 3.9]', 'Moyenne ]3.9 , 4.2]', 'Elevée ]4.2 , 4.5]', 'Excellente ]4.5 , 5]'])

# rating count --> Pour créer une variable POPULARITY
FDS['rating_count_category'] = pd.cut(FDS['rating_count'],
                           bins=[0, 50, 200, 1000, 3000, 300000],
                           labels=['Très faible (<50 votes)', 'Faible ]50 , 200] votes', 'Moyenne ]200 , 1000] votes', 'Elevée ]1000 , 3000] votes', 'Excellente (>3000 votes]'])


# main_accord --> Sous forme de liste donc + difficile à segmenter
# gender --> OK
# sillage --> OK
# longevity --> OK
# price_feeling --> OK
# top_notes --> Sous forme de liste donc + difficile à segmenter
# heart_notes --> Sous forme de liste donc + difficile à segmenter
# base_notes --> Sous forme de liste donc + difficile à segmenter
# url --> OK

#On remet la colonne 'url' à la fin
# Déplacer la colonne 'url' à la fin
cols = [col for col in FDS.columns if col != 'url'] + ['url']
FDS = FDS[cols]



## **DESACTIVE** Standardiser / scaler les variables numériques

In [11]:
from sklearn.preprocessing import StandardScaler
'''
num_cols = ["launch_year", "rating_value", "rating_count"]
scaler = StandardScaler()

# Appliquer le scaler et ajouter les colonnes scalées avec un suffixe
scaled_cols = scaler.fit_transform(FDS[num_cols])
scaled_df = pd.DataFrame(scaled_cols, columns=[f"{col}_scaled" for col in num_cols], index=FDS.index)

# Concaténer les colonnes scalées au DataFrame original
FDS = pd.concat([FDS, scaled_df], axis=1)
'''

'\nnum_cols = ["launch_year", "rating_value", "rating_count"]\nscaler = StandardScaler()\n\n# Appliquer le scaler et ajouter les colonnes scalées avec un suffixe\nscaled_cols = scaler.fit_transform(FDS[num_cols])\nscaled_df = pd.DataFrame(scaled_cols, columns=[f"{col}_scaled" for col in num_cols], index=FDS.index)\n\n# Concaténer les colonnes scalées au DataFrame original\nFDS = pd.concat([FDS, scaled_df], axis=1)\n'

## One-hot / label encoding des variables catégorielles simples 

**"gender", "longevity", "sillage", "price_feeling", "launch_period","rating_category","popularity"**

In [12]:
from sklearn.preprocessing import OneHotEncoder

simple_cat = ["gender", "longevity", "sillage", "price_feeling", "launch_period","rating_value_category","rating_count_category"]
ohe = OneHotEncoder(sparse_output=False, drop="first")          # drop pour éviter la colinéarité
ohe_mat = ohe.fit_transform(FDS[simple_cat])
ohe_FDS  = pd.DataFrame(ohe_mat, columns=ohe.get_feature_names_out(simple_cat), index=FDS.index)

FDS = pd.concat([FDS, ohe_FDS], axis=1)

# Liste des variables actuelles
new_vars = ['gender_male', 'gender_unisex', 'gender_nan', 'longevity_long lasting',
            'longevity_moderate', 'longevity_very weak', 'longevity_weak', 'longevity_nan',
            'sillage_enormous', 'sillage_intimate', 'sillage_strong', 'sillage_nan',
            'price_feeling_great value', 'price_feeling_ok', 'price_feeling_overpriced',
            'price_feeling_way overpriced', 'price_feeling_nan', 'launch_period_2000-2005',
            'launch_period_2005-2010', 'launch_period_2010-2015', 'launch_period_2015-2020',
            'launch_period_Après 2020', 'launch_period_Avant 1995', 'launch_period_nan',
            'rating_value_category_Excellente ]4.5 , 5]', 'rating_value_category_Faible ]3.3 , 3.9]',
            'rating_value_category_Moyenne ]3.9 , 4.2]', 'rating_value_category_Très faible [0 , 3.3]',
            'rating_value_category_nan', 'rating_count_category_Excellente (>3000 votes]',
            'rating_count_category_Faible ]50 , 200] votes', 'rating_count_category_Moyenne ]200 , 1000] votes',
            'rating_count_category_Très faible (<50 votes)', 'rating_count_category_nan']

# Renommer les variables
new_vars_renamed = [
    "ohe_" + var.replace("launch_period", "LP")
                 .replace("rating_value_category", "RVC")
                 .replace("rating_count_category", "RCC")
    for var in new_vars
]

# Afficher les nouvelles variables
print("Noms des variables renommées :", new_vars_renamed)

# Renommer les colonnes dans le DataFrame
FDS.rename(columns=dict(zip(new_vars, new_vars_renamed)), inplace=True)


Noms des variables renommées : ['ohe_gender_male', 'ohe_gender_unisex', 'ohe_gender_nan', 'ohe_longevity_long lasting', 'ohe_longevity_moderate', 'ohe_longevity_very weak', 'ohe_longevity_weak', 'ohe_longevity_nan', 'ohe_sillage_enormous', 'ohe_sillage_intimate', 'ohe_sillage_strong', 'ohe_sillage_nan', 'ohe_price_feeling_great value', 'ohe_price_feeling_ok', 'ohe_price_feeling_overpriced', 'ohe_price_feeling_way overpriced', 'ohe_price_feeling_nan', 'ohe_LP_2000-2005', 'ohe_LP_2005-2010', 'ohe_LP_2010-2015', 'ohe_LP_2015-2020', 'ohe_LP_Après 2020', 'ohe_LP_Avant 1995', 'ohe_LP_nan', 'ohe_RVC_Excellente ]4.5 , 5]', 'ohe_RVC_Faible ]3.3 , 3.9]', 'ohe_RVC_Moyenne ]3.9 , 4.2]', 'ohe_RVC_Très faible [0 , 3.3]', 'ohe_RVC_nan', 'ohe_RCC_Excellente (>3000 votes]', 'ohe_RCC_Faible ]50 , 200] votes', 'ohe_RCC_Moyenne ]200 , 1000] votes', 'ohe_RCC_Très faible (<50 votes)', 'ohe_RCC_nan']


**Nombre de variables ajoutées** : gender (3), longevity (5), sillage (4), price_feeling (5), launch_period (7), rvc (5), rcc (5)   **--> 34 nouvelles variables**

## MultiLabelBinarizer des variables listes en (0,1)

main_accord, top_notes, middle_notes, base_notes

### **Convertir les listes de listes main_accord, top_notes,, etc.. en liste**

In [15]:
import ast

# convertir en vraie liste
FDS["main_accords_list"] = FDS["main_accords"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
# Convertir en vraies listes pour top_notes, middle_notes, et base_notes
FDS["top_notes_list"] = FDS["top_notes"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
FDS["middle_notes_list"] = FDS["middle_notes"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
FDS["base_notes_list"] = FDS["base_notes"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])


# print du nombre de main accords unique
print(f"Nombre de main accords unique = {FDS['main_accords_list'].explode().nunique()}")
# print du nombre de  top/ middle / base notes unique
print(f"Nombre de top notes unique = {FDS['top_notes_list'].explode().nunique()}")
print(f"Nombre de middle notes unique = {FDS['middle_notes_list'].explode().nunique()}")
print(f"Nombre de base notes unique = {FDS['base_notes_list'].explode().nunique()}")

Nombre de main accords unique = 84
Nombre de top notes unique = 1024
Nombre de middle notes unique = 1138
Nombre de base notes unique = 772


### Explorer ces listes pour comprendre comment les gérer

In [31]:
MA_list = [accord for accord in FDS['main_accords_list'].explode().unique() if isinstance(accord, str)]
TN_list = [note for note in FDS['top_notes_list'].explode().unique() if isinstance(note, str)]
MN_list = [note for note in FDS['middle_notes_list'].explode().unique() if isinstance(note, str)]
BN_list = [note for note in FDS['base_notes_list'].explode().unique() if isinstance(note, str)]

# Trier par ordre alphabétique
MA_list.sort()
TN_list.sort()
MN_list.sort()
BN_list.sort()


# mettre chaque list dans un csv
pd.DataFrame(MA_list, columns=['main_accords']).to_csv('main_accords.csv', index=False)
pd.DataFrame(TN_list, columns=['top_notes']).to_csv('top_notes.csv', index=False)
pd.DataFrame(MN_list, columns=['middle_notes']).to_csv('middle_notes.csv', index=False)
pd.DataFrame(BN_list, columns=['base_notes']).to_csv('base_notes.csv', index=False)

# Analyse Top notes
substrings = ['Blue ', 'Blood ', 'Brazilian ', 'Bourbon ', 'Calabrian ', ' Seed', ' Seeds', ' Leaf', 'Red ', 
              'Sicilian ', 'Sweet ', 'White ', 'Wild ', 'Yellow ']

for substring in substrings:
    for element in TN_list:
        if substring in element:
            print(element)


Blue Hyacinth
Blue Lavender
Blue Lotus
Blue Rose
Blue Sage
Blood Grapefruit
Blood Mandarin
Blood Orange
Brazilian Mandarin
Brazilian Orange
Brazilian Redwood
Brazilian Rosewood
Bourbon Geranium
Bourbon Pepper
Bourbon Vanilla
Calabrian Mandarin
Calabrian bergamot
Carrot Seeds
Celery Seeds
Hibiscus Seed
Pimento Seeds
Carrot Seeds
Celery Seeds
Pimento Seeds
Apple Leaf
Bamboo Leaf
Banana Leaf
Basil Leaf
Bay Leaf
Bergamot Leaf
Birch Leaf
Cinnamon Leaf
Citron Leaf
Citrus Leaf
Clove Leaf
Coriander Leaf
Cypress Leaf
Fig Leaf
Galbanum Leaf
Jasmine Leaf
Lemon Leaf
Lemon Leaf Oil
Lotus Leaf
Magnolia Leaf
Mandarin Leaf
Olive Leaf
Orange Leaf
Palm Leaf
Patchouli Leaf
Pear Leaf
Pimento Leaf
Pineapple Leaf
Pistachio Leaf
Raspberry Leaf
Rhubarb Leaf
Strawberry Leaf
Tea Leaf
Tobacco Leaf
Tomato Leaf
Violet Leaf
Wild Strawberry Leaf
Red Algae
Red Apple
Red Berries
Red Chilli Pepper
Red Currant
Red Freesia
Red Fruits
Red Litchi
Red Mandarin
Red Mulberry
Red Peach
Red Peony
Red Pepper
Red Poppy
Red Rose
R

*Observations :*

- Problème de casse

- Main accord : rien à changer, regrouper, ...

- Top notes : 
    
    Supprimer : 'Blue ', 'Blood ', 'Brazilian ', 'Bourbon ', 'Calabrian ', ' Seed', ' Seeds', ' Leaf', 
    certains 'Red', sewwet pas pour tous
    
    almond, bamboo, basil, orange, iris, geranium, Grapefruit
Mandarin
Orange
supprimer 'blue_' ? blood_ , brazilian_ , bourbon_ calabrian_, _seed, _seeds _leaf, red_ Sicilian_ Sweet_ white_ wild_ yellow_
regrouper pine plum tea tonka

cardamon = cardamom

In [ ]:
# Mettre en majuscule la première lettre de chaque élément
Groupe_list = [MA_list, TN_list, MN_list, BN_list]
for liste in Groupe_list:
    liste = [item.capitalize() for item in liste]

# Regrouper certains top_notes


### **Encodage**

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

# Encodage des accords principaux
accords_encoded = pd.DataFrame(
    mlb.fit_transform(FDS["main_accords_list"]),
    columns=[f"MA_{a}" for a in mlb.classes_],
    index=FDS.index
)
# Encodage des top_notes
top_notes_encoded = pd.DataFrame(
    mlb.fit_transform(FDS["top_notes_list"]),
    columns=[f"TN_{note}" for note in mlb.classes_],
    index=FDS.index
)
# Encodage des middle_notes
middle_notes_encoded = pd.DataFrame(
    mlb.fit_transform(FDS["middle_notes_list"]),
    columns=[f"MN_{note}" for note in mlb.classes_],
    index=FDS.index
)
# Encodage des base_notes
base_notes_encoded = pd.DataFrame(
    mlb.fit_transform(FDS["base_notes_list"]),
    columns=[f"BN_{note}" for note in mlb.classes_],
    index=FDS.index
)

# Ajouter les colonnes encodées au DataFrame principal et supprimer les colonnes originales
FDS = pd.concat([FDS, accords_encoded], axis=1).drop(columns=["main_accords_list"])
FDS = pd.concat([FDS, top_notes_encoded, middle_notes_encoded, base_notes_encoded], axis=1).drop(
    columns=["top_notes_list", "middle_notes_list", "base_notes_list"]
)


Nombre de main accords unique = 56
Nombre de top notes unique = 97
Nombre de middle notes unique = 88
Nombre de base notes unique = 41


,name,brand,nose,launch_year,rating_value,rating_count,main_accords,gender,longevity,sillage,...,BN_Sandalwood,BN_Smoke,BN_Tonka Bean,BN_Vanilla,BN_Vanille,BN_Vetiver,BN_Whipped Cream,BN_White Musk,BN_Woodsy Notes,BN_Woody Notes
2526,Love Spell Radiant,Victoria's Secret,nan,2021.0,3.96,25,"['tropical', 'citrus', 'sweet', 'fruity', 'ter...",female,weak,average,...,0,0,0,0,0,0,0,0,0,0
14959,Fresh Sparkling Snow,Bath & Body Works,nan,2015.0,4.10,82,"['fresh', 'fruity', 'aquatic', 'ozonic', 'flor...",female,moderate,average,...,0,0,0,0,0,0,0,0,0,0
13474,Hugo Urban Journey,Hugo Boss,nan,2018.0,3.65,142,"['green', 'woody', 'floral', 'fresh']",male,moderate,average,...,1,0,0,0,0,0,0,0,0,0
7502,Dr. Botica Poção da Coragem,O Boticário,nan,2018.0,3.90,29,"['aromatic', 'marine', 'fresh spicy', 'woody',...",unisex,moderate,intimate,...,0,0,0,0,0,0,0,0,0,0
12542,Idylle,Guerlain,Thierry Wasser,2009.0,3.79,6705,"['white floral', 'floral', 'fresh', 'rose', 'm...",female,moderate,average,...,0,0,0,0,0,0,0,0,0,0


# Sauvegarder le jdd propre

Réorganiser les colonnes

In [67]:
# Liste des colonnes initiales
columns_order = [
    "name", "brand", "brand_reduce", "nose", "nose_reduce",
    "launch_year", "launch_period", "rating_value", 'rating_value_category', "rating_count", "rating_count_category",
    "main_accords", "gender", "longevity", "sillage", "price_feeling",
    "top_notes", "middle_notes", "base_notes", "launch_period",
    "url"
]

# Réorganiser les colonnes dans FDS
FDS = FDS[[col for col in columns_order if col in FDS.columns] + 
          [col for col in FDS.columns if col not in columns_order]]

Mettre le type string aux bonnes colonnes

In [68]:
FDS['name'] = FDS['name'].astype(str)
FDS['brand'] = FDS['brand'].astype(str)
FDS['brand_reduce'] = FDS['brand_reduce'].astype(str)
FDS['nose'] = FDS['nose'].fillna("Non connu").replace("nan", "Non connu").astype(str)
FDS['nose_reduce'] = FDS['nose_reduce'].fillna("Non connu").replace("nan", "Non connu").astype(str)


In [69]:
FDS.to_csv("FDS_clean.csv", index=False)
print("✅  Dataset nettoyé et encodé enregistré → parfums_clean.csv")


✅  Dataset nettoyé et encodé enregistré → parfums_clean.csv


**Mieux comprendre les nuvelles variables**

*Variables basiques* :
- Numériques : "launch_year", "rating_value", "rating_count"
- catégorielles : "name", "brand", "nose", "gender", "longevity", "sillage", "price_feeling"
- Listes : "main_accords", "top_notes", "middle_notes", "base_notes",
- Autres : "url"

*Variables segmentées* :
- catégorielles :"brand_reduce", "nose_reduce", "launch_period", 'rating_value_category', "rating_count_category"

*Variables encodées* :
- OHE : gender (3), longevity (5), sillage (4), price_feeling (5), launch_period (7), rvc (5), rcc (5)   **--> 34**

- Multi Encode : MA main_accord, TN top_note, MN middle_note, BN base_note 

Nombre de main accords unique = 84

Nombre de top notes unique = 1024

Nombre de middle notes unique = 1138

Nombre de base notes unique = 772